Import data and libraries

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import quantile_transform
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import OneHotEncoder  ##. better to use dummy from pandas
from sklearn.preprocessing import PowerTransformer
from scipy.stats import boxcox
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from scipy.stats import boxcox
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

pd.options.display.max_rows = 100

baseDf = pd.read_csv('../../data/helathcareforall.csv')

In [2]:
baseDf

,CONTROLN,STATE,GENDER,HV1,IC1,IC4,HVP1,IC5,POBC1,POBC2,IC2,IC3,AVGGIFT,TCODE,DOB,DOMAIN,TARGET_D
0,44060,FL,M,AAA896,392,520.0,7,21975,6,16,430.0,466,28.000000,1,1901,C2,100.0
1,96093,IL,M,537.00,365,473.0,0,19387,1,89,415.0,410,5.666667,0,0,T2,7.0
2,43333,FL,F,725.00,301,436.0,3,18837,11,17,340.0,361,4.111111,0,2501,C2,5.0
3,21885,NC,M,AAA1095,401,413.0,7,14014,1,74,407.0,399,27.277778,0,2208,T2,38.0
4,190108,FL,F,995.00,252,348.0,0,17991,5,6,280.0,316,6.000000,28,0,C2,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,114721,OK,NaN,1040,472,656.0,0,26962,2,56,609.0,579,11.666667,0,0,C2,15.0
1003,149152,CA,M,4507,842,962.0,95,54642,17,50,1004.0,893,20.000000,1,4401,S1,20.0
1004,959,IA,F,586,349,465.0,1,15304,1,77,413.0,404,7.300000,0,4404,C2,10.0
1005,179563,WA,male,842,420,494.0,2,12894,41,24,419.0,476,16.400000,1,5001,S2,23.0


split numerical from categorical

In [3]:
numerical = baseDf.select_dtypes(include=np.number)
categorical = baseDf.select_dtypes(include=object)

##Linear Regression

Train-test split.

In [26]:
X = numerical.drop('TARGET_D', axis=1)
y = numerical.TARGET_D
X = X._get_numeric_data()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30, random_state=123)
# X_train.fillna(0)

,CONTROLN,IC1,IC4,HVP1,POBC1,POBC2,IC2,IC3,AVGGIFT,TCODE,DOB
591,90031,417,540.0,2,11,71,472.0,494,3.722222,2,1601
178,15668,383,433.0,0,0,84,400.0,398,9.333333,0,4101
841,102351,208,356.0,0,0,59,271.0,264,13.333333,0,3301
243,54142,210,310.0,0,0,55,254.0,272,13.000000,2,4601
944,35325,277,387.0,0,8,21,350.0,329,12.142857,0,1801
...,...,...,...,...,...,...,...,...,...,...,...
988,66223,319,398.0,3,2,83,345.0,357,9.571429,0,5801
322,10973,563,660.0,75,8,59,579.0,655,10.894737,1,2305
382,16099,282,335.0,1,0,81,291.0,327,5.500000,0,2701
365,152527,289,402.0,37,9,50,377.0,339,23.000000,28,4501


Standardize the data (after the data split).

In [17]:
std_scaler = StandardScaler().fit(X_train)  ##. finding the parameters ( mean, variance from the training set )
X_train_scaled = std_scaler.transform(X_train)
X_train.shape
X_test_scaled = std_scaler.transform(X_test)
X_test_scaled

array([[-0.42736901, -0.9024371 , -1.04761825, ..., -0.90580767,
        -0.18165298, -0.49718631],
       [-1.47782996, -0.59790459, -0.57751928, ...,  0.3648657 ,
        -0.18165298,  0.30945377],
       [ 0.58367308, -0.71971759, -0.8399001 , ...,  0.38180288,
        -0.18165298, -1.20940205],
       ...,
       [-1.15946276,  0.1606582 ,  0.30801598, ..., -0.57091345,
        -0.18165298,  0.87884677],
       [-0.98982459, -0.81384582, -1.11321345, ...,  1.0169471 ,
        -0.18165298, -1.20940205],
       [ 1.48149738, -1.10730442, -1.23347133, ..., -0.81589765,
        -0.17739047, -0.96788452]])

Apply linear regression.

In [27]:

   # model train

# X_train_const_scaled = sm.add_constant(X_train_scaled)  # adding a constant
#
# # model = sm.OLS(y_train, X_train_const_scaled).fit()
# predictions_train = model.predict(X_train_const_scaled)
# predictions_train
# # X_test_const_scaled = sm.add_constant(X_test_scaled)  # adding a constant test data
# # predictions_test = model.predict(X_test_const_scaled)
# # print_model = model.summary()
# model = LinearRegression()  # model
# # model.fit(X_train_scaled, y_train)  # model train

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

Model Interpretation.